In [27]:
# This is the program for heuristic optimisation of IC.
import numpy as np
from scipy.optimize import minimize_scalar, differential_evolution
import warnings

warnings.filterwarnings("ignore")

# --- CORE MATH UTILITIES ---
def binary_entropy(p):
    p = np.clip(p, 1e-20, 1 - 1e-20)
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

def kl_divergence(p, q, eps=1e-15):
    p, q = np.clip(p, eps, 1.0), np.clip(q, eps, 1.0)
    return np.sum(p * np.log2(p / q))

def get_projected_info(p_ab_target, p_ab_source):
    p_a = np.sum(p_ab_target, axis=1)
    p_b_src = np.sum(p_ab_source, axis=0)
    p_b_tgt = np.sum(p_ab_target, axis=0)
    p_a_cond_src = p_ab_source / (p_b_src + 1e-15)
    p_a_cond_tgt = p_ab_target / (p_b_tgt + 1e-15)
    
    pi_sum = 0
    for b in range(2):
        obj = lambda l: kl_divergence(p_a_cond_tgt[:, b], l*p_a_cond_src[:, 0] + (1-l)*p_a_cond_src[:, 1])
        res = minimize_scalar(obj, bounds=(0, 1), method='bounded')
        q_star = res.x * p_a_cond_src[:, 0] + (1 - res.x) * p_a_cond_src[:, 1]
        for a in range(4):
            if p_ab_target[a, b] > 1e-15:
                pi_sum += p_ab_target[a, b] * np.log2(q_star[a] / p_a[a])
    return pi_sum

# --- EXTREMAL PHYSICS ---
def get_steered_angle(a_x, theta, s):
    multiplier = np.tan(theta) if s == 1 else (1.0 / np.tan(theta))
    return 2 * np.arctan(np.tan(a_x / 2.0) * multiplier)

def get_alternating_violation(theta, a0, a1, b0, b1):
    best_v = float('inf')
    for s in [1, -1]:
        at0 = get_steered_angle(a0, theta, s) % np.pi
        for t in [1, -1]:
            at1 = get_steered_angle(a1, theta, t) % np.pi
            v = (max(0, -at0) + max(0, at0 - b0) + max(0, b0 - at1) + max(0, at1 - b1))
            if v < best_v: best_v = v
    return best_v

def get_p_ab_xy(theta, a0, a1, b0, b1):
    c, s = np.cos(theta), np.sin(theta)
    rho = np.array([[c*c, 0, 0, c*s], [0, 0, 0, 0], [0, 0, 0, 0], [c*s, 0, 0, s*s]])
    def get_op(angle):
        m = np.array([[np.cos(angle), np.sin(angle)], [np.sin(angle), -np.cos(angle)]])
        return [0.5 * (np.eye(2) + m), 0.5 * (np.eye(2) - m)]
    A = [get_op(a0), get_op(a1)]; B = [get_op(b0), get_op(b1)]
    P = np.zeros((2, 2, 2, 2))
    for x in range(2):
        for y in range(2):
            for a in range(2):
                for b in range(2):
                    P[a, b, x, y] = np.real(np.trace(rho @ np.kron(A[x][a], B[y][b])))
    return P

# --- MULTI-VERTEX OBJECTIVE ---
def objective(params, pc, n_v):
    # 1. Weights extraction (normalized to sum to 1)
    raw_w = params[:n_v-1]
    weights = []
    rem = 1.0
    for w_val in raw_w:
        actual_w = w_val * rem
        weights.append(actual_w)
        rem -= actual_w
    weights.append(rem)
    
    # 2. Mixing boxes from strictly extremal points
    p_mix = np.zeros((2, 2, 2, 2))
    penalty = 0
    for i in range(n_v):
        start_idx = (n_v - 1) + (i * 5)
        p_params = params[start_idx : start_idx + 5]
        v = get_alternating_violation(*p_params)
        penalty += v
        p_mix += weights[i] * get_p_ab_xy(*p_params)
    
    if penalty > 1e-5:
        return 20.0 + penalty # High penalty for non-extremal vertices

    # 3. IC_red Protocol
    p_a_bi = np.zeros((2, 4, 2))
    for a_val in range(4):
        a1, a2 = (a_val >> 1) & 1, a_val & 1
        x = a1 ^ a2
        for i in range(2):
            for alpha in range(2):
                for beta in range(2):
                    prob = 0.25 * p_mix[alpha, beta, x, i]
                    m = alpha ^ a1
                    p_a_bi[i, a_val, beta ^ m] += prob * pc
                    p_a_bi[i, a_val, beta ^ m ^ 1] += prob * (1-pc)

    mis = [0, 0]
    for i in range(2):
        pj = p_a_bi[i]; pa, pb = np.sum(pj, axis=1), np.sum(pj, axis=0)
        for a in range(4):
            for b in range(2):
                if pj[a, b] > 1e-15:
                    mis[i] += pj[a, b] * np.log2(pj[a, b] / (pa[a] * pb[b]))
        
    ir = min(get_projected_info(p_a_bi[0], p_a_bi[1]), get_projected_info(p_a_bi[1], p_a_bi[0]))
    return -(sum(mis) - ir)

# ... [Keep your existing code above the if __name__ == "__main__": line] ...

if __name__ == "__main__":
    pc_target = 0.5001
    k_cap = 1 - binary_entropy(pc_target)
    
    # Choose number of points to mix
    n_v = 5 
    print(f"Searching for IC_red violation using {n_v} mixed correlations...")
    
    # Bounds: (n_v - 1) weights + (n_v * 5) physical parameters
    bounds = [(0, 1)]*(n_v - 1) + [(0.01, np.pi/2.1), (0, np.pi), (0, np.pi), (0, np.pi), (0, np.pi)]*n_v
    
    # disp=False hides the iteration/convergence logs
    res = differential_evolution(
    objective, bounds, args=(pc_target, n_v), 
    popsize=20,       # Increase population for better global coverage
    tol=1e-16,        # Tighten tolerance significantly
    atol=1e-16,       # Absolute tolerance is key for small values
    disp=False, 
    seed=42
)

    # --- CALCULATIONS ---
    ic_red_val = -res.fun 
    violation = max(0, ic_red_val - k_cap)

    # Reconstruct Weights
    raw_w = res.x[:n_v-1]
    weights = []
    rem = 1.0
    for w_val in raw_w:
        actual_w = w_val * rem
        weights.append(actual_w)
        rem -= actual_w
    weights.append(rem)

    # --- FINAL OUTPUTS ---
    print("\n" + "="*60)
    print(f"{'OPTIMIZED RESULTS':^60}")
    print("="*60)
    print(f"{'Optimized IC_red value:':<35} {ic_red_val:.8e}")
    print(f"{'Channel Capacity (k):':<35} {k_cap:.8e}")
    print(f"{'Maximum IC Violation:':<35} {violation:.8e}")
    print("-" * 60)
    
    print("Optimized Mixing Weights:")
    for i, w in enumerate(weights):
        print(f"  - Weight {i+1}: {w:.6f}")
    
    print("\nOptimized Parameters per Correlation (theta, a0, a1, b0, b1):")
    for i in range(n_v):
        start_idx = (n_v - 1) + (i * 5)
        p = res.x[start_idx : start_idx + 5]
        param_fmt = ", ".join([f"{val:.4f}" for val in p])
        print(f"  - Corr {i+1}: ({param_fmt})")
    
    print("="*60)
    

Searching for IC_red violation using 5 mixed correlations...

                     OPTIMIZED RESULTS                      
Optimized IC_red value:             2.88539035e-08
Channel Capacity (k):               2.88539010e-08
Maximum IC Violation:               2.44265540e-15
------------------------------------------------------------
Optimized Mixing Weights:
  - Weight 1: 1.000000
  - Weight 2: 0.000000
  - Weight 3: 0.000000
  - Weight 4: 0.000000
  - Weight 5: 0.000000

Optimized Parameters per Correlation (theta, a0, a1, b0, b1):
  - Corr 1: (0.7854, 1.2938, 1.2972, 1.2956, 2.8180)
  - Corr 2: (1.0145, 1.9219, 1.6516, 1.6908, 2.5238)
  - Corr 3: (0.6167, 0.7265, 1.3477, 1.0212, 1.2286)
  - Corr 4: (1.0014, 0.8735, 1.1326, 0.8732, 3.1004)
  - Corr 5: (0.4901, 0.6021, 1.3233, 1.1998, 2.8359)
